In [1]:
import pandas as pd
import sklearn.metrics as metrics

from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

#ignore minor warnings
import warnings
warnings.filterwarnings("ignore")

import acquire
import prepare
import model

In [14]:
df_in = acquire.get_data(query_url = False)
print(df_in.shape)
df_in = prepare.prepare_dataframe(df_in)
df_in = model.prepare_model_df(df_in.sample(10_000)) # DO NOT USE THIS LINE IF NOT MODELING
train, validate, test = prepare.split_data(df_in)
train.shape, validate.shape, test.shape

Found file in working directory!
(70932, 83)


((7500, 2056), (1250, 2056), (1250, 2056))

In [15]:
outputs = model.model_loop(train, validate)
pd.DataFrame(outputs)

,model,train_RMSE,train_r2,train_explained_variance,validate_RMSE,validate_r2,validate_explained_variance
0,baseline_mean,9.683582e+01,0.000000,0.000000e+00,5.029580e+01,-1.214021e-04,0.000000e+00
1,baseline_median,9.704538e+01,-0.004333,2.220446e-16,5.062838e+01,-1.339161e-02,0.000000e+00
2,DecisionTreeRegressor,2.104099e-18,1.000000,1.000000e+00,3.522167e+01,5.095336e-01,5.112054e-01
3,LinearSVR,9.599620e+01,0.017266,2.019892e-02,4.940200e+01,3.510875e-02,4.383080e-02
4,LinearRegression,6.993198e+01,0.478470,4.784698e-01,4.509405e+12,-8.039477e+21,-8.021155e+21
5,LassoLars,9.683582e+01,0.000000,0.000000e+00,5.029580e+01,-1.214021e-04,0.000000e+00
6,RadiusNeighborsRegressor,9.608015e+01,0.015546,1.585008e-02,4.955736e+01,2.903047e-02,2.946928e-02
7,RadiusNeighborsRegressor_weighted,0.000000e+00,1.000000,1.000000e+00,4.938042e+01,3.595148e-02,3.649866e-02
8,KNeighborsRegressor,6.371402e+01,0.567090,5.671586e-01,4.240108e+01,2.892070e-01,2.899770e-01
9,KNeighborsRegressor_weighted,0.000000e+00,1.000000,1.000000e+00,4.079888e+01,3.419093e-01,3.426754e-01


In [4]:
# train.select_dtypes(include=float).info()

In [5]:
X_train, y_train, X_val, y_val = model.scale_and_make_X_and_y(train, validate)

In [6]:
rn = RadiusNeighborsRegressor(radius=5)

In [7]:
rn = rn.fit(X_train, y_train['usd'])

In [8]:
y_train['predicted'] = rn.predict(X_train)

In [9]:
y_train

,id,name,usd,predicted
13741,315f1cf8-7bde-4592-a14e-ef9e2e3e7ac0,Distortion Strike,0.38,3.469328
65371,eb981f8b-4f03-4fd2-a255-b10635958bfd,Kitsune Palliator,0.17,3.475947
54613,c4de778a-4419-49a5-9dcf-5d0095c873fa,Longtusk Cub,0.08,2.060981
58980,d45894ad-3514-44e6-882c-03fa9fd12c11,Blasted Landscape,1.04,17.395793
18004,40ef45e3-b12a-403e-823c-44f4c95543e8,Irencrag Pyromancer,1.44,2.031614
...,...,...,...,...
10224,24ba0b83-9671-4ee7-996d-57a3616b9c66,Fatal Lore,1.20,12.998587
56655,cc025f33-1b1a-428f-be28-02d7580a0a8a,Lightning Greaves,6.56,10.443497
64852,e99ee7b4-7a9b-4ccf-830b-adf203e2fe7f,Bogardan Hellkite,0.31,4.434998
29984,6c3b29dd-2c2f-492d-aa23-140d44152bd9,Plagued Rusalka,0.08,2.819914


In [10]:
# metrics.mean_squared_error(y_t['usd'], y_v['predicted'])
y_val['predicted'] = rn.predict(X_val) #why is it predicting only NaN?

In [11]:
y_val

,id,name,usd,predicted
34000,7b1967f0-f6a9-469a-96f9-57fe77059e88,Skull Catapult,0.14,15.219120
52967,bee37df6-ca09-44f2-b4a4-ad21be4b2a4d,Fugue,0.15,7.110022
58509,d286f664-86e9-4d5a-8cd0-e162cb7d9791,Render Silent,2.06,7.621836
12730,2dd0fef1-209f-4de5-a736-8f9bca2faa0a,Fang of Shigeki,0.07,1.091728
3479,0c4eaecf-dd4c-45ab-9b50-2abe987d35d4,Island,6.95,2.080187
...,...,...,...,...
16148,39f322ff-0b04-41ce-90cd-9896f941e703,Urborg Justice,12.01,13.022484
35157,7f5e6a5a-18ee-438f-ab16-694e19a2410e,Azra Oddsmaker,0.59,5.081174
2689,097f0f7a-0a2d-47bc-8c9c-aaad0a9d8f19,Forest,0.82,2.518791
12266,2c211958-1ee2-4dda-9ef1-c59de599ebc1,Veiling Oddity,0.01,1.906983


In [12]:
model.evaluate_train_validate_model(y_train, y_val, 'radius')

{'model': 'radius',
 'train_RMSE': 148.53805494627076,
 'train_r2': 0.011808519666704487,
 'train_explained_variance': 0.011991782132503626,
 'validate_RMSE': 49.765607084664325,
 'validate_r2': 0.02780677947897847,
 'validate_explained_variance': 0.02780776944024621}

In [13]:
outputs = []
outputs.append(model.make_linearsvr_model(X_train, y_train, X_val, y_val, return_predictions = False))

AttributeError: module 'model' has no attribute 'make_linearsvr_model'

In [ ]:
pd.DataFrame(outputs)

In [ ]:
sum_of_notnull = X_val.notnull().sum()
sum_of_notnull

In [ ]:
# def extract_legality_columns(df, legality_to_abstract):
#     df[f"{legality_to_abstract}_legality"] = df['legalities'].apply(lambda r : r[legality_to_abstract])
#     return df

# def get_legality_types(df):
#     game_legalities = df.legalities.iloc[0].keys()
#     df_new = pd.DataFrame()
#     for game_legality in game_legalities:
#         df_new = extract_legality_columns(df, game_legality)
#     return df_new, game_legalities

model_train = model.prepare_model_df(train)
model_validate = model.prepare_model_df(validate)
X, y, _, _ = model.scale_and_make_X_and_y(model_train, model_validate)

In [ ]:
X.select_dtypes(include=int).info()

In [ ]:
train.penny_rank.hist(bins = 200)

In [ ]:
train.penny_rank.describe()

In [ ]:
train[train.id == 'd2e9decf-47b7-44e0-b380-8055b6011021']